In [1]:
from dotenv import load_dotenv
from langchain_ollama import ChatOllama

my_llm = ChatOllama(
    base_url="http://localhost:11434",
    model="qwen3:latest",
    temperature=0.7,
    max_tokens=400
)

load_dotenv()

True

## Tool Binding with LLM

### Importing External Tool - Wikipedia

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wiki_tool = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(top_k_results=2), 
    description="This tool can provide the real-time or even the future related information.")

### Building Custom Tool - Math Functions

In [ ]:
from langchain_core.tools import tool

@tool
def add_numbers(a: int, b: int) -> int:
    "Result: Addition of 2 numbers"
    return a+b

@tool
def subtract_numbers(a: int, b: int) -> int:
    "Result: Subtraction of 2 numbers"
    return a-b

@tool
def multiply_numbers(a: int, b: float) -> float:
    "Result: Multiplication of 2 numbers"
    return a*b

@tool
def divide_numbers(a: float, b: float) -> float:
    "Result: Division of 2 numbers"
    return (a/b)


toolkit = [wiki_tool, add_numbers, subtract_numbers, multiply_numbers, divide_numbers]

## Building the Agent

In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain_core.prompts import ChatPromptTemplate

agent = initialize_agent(
    tools=toolkit,
    llm=my_llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# query = "My salary is 10K. Now I got an increment of 15%. What is new salary ?"
# query = "Who is the winner of IPL 2026 ?"
# query = "Who is the winner of the ICC Champions Trophy 2025 ? What is my salary after getting 15% raise on 10K ?"
# query = "Did Donald Trump win in the 2024 presedential election ?"

input_prompt = ChatPromptTemplate.from_messages([
    "system", "This is year 2025. You provide a concise response to the human query."
    "user", "Who is the latest winner of the ICC Champions Trophy ? What is my salary after getting 15% raise on 10K ?"
])

response = agent.invoke(input_prompt)

print(response['output'])

## Using Playwright Browser Tool with Agent

In [ ]:
%pip install --upgrade --quiet  playwright
%pip install --upgrade --quiet  lxml

### Setup the PlayWright Toolkit

In [ ]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import (
    create_async_playwright_browser
)

# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio

nest_asyncio.apply()

In [ ]:
# Creating the async browser to perform the actions
async_browser = create_async_playwright_browser()

browser_toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)

list_of_tools = browser_toolkit.get_tools()

list_of_tools


### Setup the Agent for PlayWright

In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain_core.prompts import ChatPromptTemplate

agent = initialize_agent(
    tools=list_of_tools,
    llm=my_llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

query = "What is the address of Ramakrishna Math Pune on the https://belurmath.org/ramakrishna-math-pune/ page ?"
# query = "What are the hyperlinks on the https://belurmath.org/ramakrishna-math-pune/ page ?"

result = await agent.ainvoke(query)

result['output']